In [1]:
import psycopg2
import requests

In [9]:
with open('/Users/crupley/.api/walkscore.txt') as f:
    wskey = f.readline().strip()

In [ ]:
lat = 37.7700103
lon = -122.4535951

In [60]:
requrl = 'http://api.walkscore.com/score'
payload = {'wsapikey':wskey,
           'lat': lat,
           'lon': lon,
           'format': 'json'}
response = requests.get(requrl, params = payload)

In [15]:
response.json()

{u'description': u"Walker's Paradise",
 u'help_link': u'https://www.redfin.com/how-walk-score-works',
 u'logo_url': u'https://cdn.walk.sc/images/api-logo.png',
 u'more_info_icon': u'https://cdn.walk.sc/images/api-more-info.gif',
 u'more_info_link': u'https://www.redfin.com/how-walk-score-works',
 u'snapped_lat': 37.77,
 u'snapped_lon': -122.454,
 u'status': 1,
 u'updated': u'2015-06-01 22:34:36.166320',
 u'walkscore': 93,
 u'ws_link': u'https://www.walkscore.com/score/loc/lat=37.7700103/lng=-122.4535951/?utm_source=crupley.com&utm_medium=ws_api&utm_campaign=ws_api'}

In [61]:
data = pd.Series(response.json())
data['searched_lat'] = lat
data['searched_lon'] = lon
data

help_link               https://www.redfin.com/how-walk-score-works
logo_url                    https://cdn.walk.sc/images/api-logo.png
more_info_icon         https://cdn.walk.sc/images/api-more-info.gif
more_info_link          https://www.redfin.com/how-walk-score-works
snapped_lat                                                  -0.051
snapped_lon                                                 -0.0645
status                                                            2
ws_link           https://www.walkscore.com/score/loc/lat=-0.050...
searched_lat                                              -0.050653
searched_lon                                             -0.0640791
dtype: object

In [27]:
for i in list(data.keys()): print i

description
help_link
logo_url
more_info_icon
more_info_link
snapped_lat
snapped_lon
status
updated
walkscore
ws_link
searched_lat
searched_lon


In [35]:
data.values

array([u"Walker's Paradise",
       u'https://www.redfin.com/how-walk-score-works',
       u'https://cdn.walk.sc/images/api-logo.png',
       u'https://cdn.walk.sc/images/api-more-info.gif',
       u'https://www.redfin.com/how-walk-score-works', 37.77, -122.454, 1,
       u'2015-06-01 22:34:36.166320', 93,
       u'https://www.walkscore.com/score/loc/lat=37.7700103/lng=-122.4535951/?utm_source=crupley.com&utm_medium=ws_api&utm_campaign=ws_api',
       37.7700103, -122.4535951], dtype=object)

In [36]:
ncols = data.shape[0]
q_string = 	'''
    INSERT INTO walkscore_raw (description,
                               help_link,
                               logo_url,
                               more_info_icon,
                               more_info_link,
                               snapped_lat,
                               snapped_lon,
                               status,
                               updated,
                               walkscore,
                               ws_link,
                               searched_lat,
                               searched_lon)
        VALUES (%s)''' % ('%s, ' * (ncols - 1) + '%s')

In [38]:
conn = psycopg2.connect(dbname='hoodie', user='postgres', host='/tmp')
c = conn.cursor()

c.execute(q_string, data.values)

conn.commit()
conn.close()

In [63]:
%run ../code/geo.py

In [64]:
print g.shape
g[:5,:]

(5476, 2)


array([[  37.719347  , -122.51807913],
       [  37.72071748, -122.51807913],
       [  37.72208796, -122.51807913],
       [  37.72345844, -122.51807913],
       [  37.72482893, -122.51807913]])

In [54]:
%run ../code/scrape.py

In [66]:
for lat, lon in g[5:,:]:
     scrape_walkscore(lat, lon)

# greatschools

In [69]:
with open('/Users/crupley/.api/greatschools.txt') as f:
    gskey = f.readline().strip()

In [96]:
state = 'CA'
city = 'San-Francisco'

requrl = 'http://api.greatschools.org/schools/' + state + '/' + city
payload = {'key': gskey,
           'limit': '-1'}

In [97]:
response = requests.get(requrl, params = payload)

In [79]:
response.text

u'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><schools><school><gsId>25079</gsId><name>Academy Of Arts And Sciences</name><type>public</type><gradeRange>9-12</gradeRange><enrollment>341</enrollment><gsRating>6</gsRating><parentRating>2</parentRating><city>San Francisco</city><state>CA</state><districtId>717</districtId><district>San Francisco Unified School District</district><districtNCESId>0634410</districtNCESId><address>555 Portola Drive, \nSan Francisco, CA  94131</address><phone>(415) 695-5700</phone><fax>(415) 695-5326</fax><website>http://www.academysf.org</website><ncesId>063441012526</ncesId><lat>37.7459</lat><lon>-122.44999</lon><overviewLink>http://www.greatschools.org/california/san-francisco/25079-Academy-Of-Arts-And-Sciences/?s_cid=gsapi</overviewLink><ratingsLink>http://www.greatschools.org/school/rating.page?state=CA&amp;id=25079&amp;s_cid=gsapi</ratingsLink><reviewsLink>http://www.greatschools.org/school/parentReviews.page?state=CA&amp;id=25079&amp;s_cid=gs

In [98]:
from xml.etree import ElementTree

In [99]:
tree = ElementTree.fromstring(response.content)

In [100]:
print tree.tag
print tree.attrib

schools
{}


In [101]:
len(tree)

200

In [102]:
tree[0][0].tag

'gsId'

In [120]:
for att in tree[0]:
    print att.tag, att.text

gsId 25079
name Academy Of Arts And Sciences
type public
gradeRange 9-12
enrollment 341
gsRating 6
parentRating 2
city San Francisco
state CA
districtId 717
district San Francisco Unified School District
districtNCESId 0634410
address 555 Portola Drive, 
San Francisco, CA  94131
phone (415) 695-5700
fax (415) 695-5326
website http://www.academysf.org
ncesId 063441012526
lat 37.7459
lon -122.44999
overviewLink http://www.greatschools.org/california/san-francisco/25079-Academy-Of-Arts-And-Sciences/?s_cid=gsapi
ratingsLink http://www.greatschools.org/school/rating.page?state=CA&id=25079&s_cid=gsapi
reviewsLink http://www.greatschools.org/school/parentReviews.page?state=CA&id=25079&s_cid=gsapi
schoolStatsLink http://www.greatschools.org/modperl/achievement/CA/25079


In [142]:
d = dict()
for school in tree:
    for att in school:
        value = att.text
        if type(value) == None: value = -1
        d[att.tag] = d.get(att.tag, list()) + [value]
d

{'address': ['555 Portola Drive, \nSan Francisco, CA  94131',
  '180 Fair Oaks Street, \nSan Francisco, CA  94110',
  '250 23rd Avenue, \nSan Francisco, CA  94121',
  '479 Miller Avenue, \nSouth San Francisco, CA  94080',
  '2265 3rd St, \nSan Francisco, CA  94107',
  '1245 Folsom Street, \nSan Francisco, CA  94105',
  '735 Fell St, \nSan Francisco, CA  94117',
  '3741 Buchanan St, \nSan Francisco, CA  94123',
  '1245 folsom st, \nSan Francisco, CA  94103',
  '300 4th St, \nSan Francisco, CA  94117',
  '116 Romney Avenue, \nSouth San Francisco, CA  94080',
  '450 Somerset Street, \nSan Francisco, CA  94134',
  '2265 3rd Street, \nSan Francisco, CA  94107',
  '625 Douglas Street, \nSan Francisco, CA  94114',
  '3627 Broderick Street, \nSan Francisco, CA  94123',
  '105 Aptos Avenue, \nSan Francisco, CA  94127',
  '175 Phelan Avenue, \nSan Francisco, CA  94112',
  '680 18th Avenue, \nSan Francisco, CA  94121',
  '555 Portola Drive, \nSan Francisco, CA  94131',
  '825 Southwood Drive, \nS

In [144]:
for key in d.keys():
    print key, len(d[key])

schoolStatsLink 200
city 200
ratingsLink 200
district 109
parentRating 167
lon 200
gsId 200
state 200
type 200
website 200
overviewLink 200
fax 200
gsRating 94
address 200
lat 200
name 200
phone 200
reviewsLink 200
enrollment 177
districtId 109
districtNCESId 109
ncesId 200
gradeRange 200


In [156]:
print school.items()

[]


[]